---
title: "nycflights13 3조 보고서(비행)"
author: '3조'
format:
    revealjs:
        logo: pa.jpg
        title-slide-attributes:
            data-background-image: background.jpg
        reference-location: document
        footer: 'LS 빅데이터 스쿨 3조 ( 김보경 / 박한슬 / 송성필 / 양현준 )'
  
      
echo: true
---

## 주제 선정 이유 
- 1. 모든 데이터가 3개의 공항에서 생성되었다.
JFK(John F. Kennedy International Airport)  
LGA(LaGuardia Airport)  
EWR(Newark Liberty International Airport)
- 2. 출발·도착의 지연이 발생한다면 특정 기간에  
과도하게 발생되는 특징이 있을 것이다.
- 3. 상기의 결과값에  
뉴욕지역의 기후도 영향을 줄 수 있다.

---

**part1**
<br>
<br>
1-1 월별 출발·도착 지연 분석
1-2 월별 출발·도착 지연과 날씨의 관계

## 1-1 월별 출발·도착 지연 최빈값

In [ ]:
#| echo: true
#| code-fold: true
from nycflights13 import flights, weather
import pandas as pd
# flights에서 특정 컬럼 추출
picked = flights[['arr_delay', 'dep_delay', 'time_hour']]
# picked['time_hour']의 타입 변환 (object -> datetime64)
picked['time_hour'] = pd.to_datetime(picked['time_hour'])
# 월정보 추출
picked['month'] = picked['time_hour'].dt.month
# 도착 지연 여부[True, False]
picked['arr_delay_flag'] = picked['arr_delay'] > 0
# 출발 지연 여부[True, False]
picked['dep_delay_flag'] = picked['dep_delay'] > 0
# 월별 지연 횟수 집계 [도착지연, 출발지연 True값의 합]
monthly_delays = picked.groupby('month').agg(
    arrival_delays=('arr_delay_flag', 'sum'),
    departure_delays=('dep_delay_flag', 'sum')
).reset_index()
print(monthly_delays)

## ** 도식화**

In [ ]:
#| echo: true
#| code-fold: true
#| eval: true
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# X 축 위치 조정[동시에 출발·도착 지연 표기 위해]
x = np.array(monthly_delays['month'])  # 월 (1~12)나열
width = 0.4  # 막대 너비
# 그래프 그리기 (도착 지연은 왼쪽, 출발 지연은 오른쪽)
plt.figure(figsize=(10, 5))
plt.bar(x - width/2, monthly_delays['arrival_delays'],
         width=width, label='Arrival Delays', color='blue', alpha=0.7)
plt.bar(x + width/2, monthly_delays['departure_delays'],
         width=width, label='Departure Delays', color='red', alpha=0.7)
plt.ticklabel_format(style='plain', axis='y')
plt.ylim(6000, 16000)
plt.xlabel('Month')
plt.ylabel('Number of Delays')
plt.title('Total montly Flight Delays')
plt.xticks(ticks=range(1, 13))  # 1~12월 표시
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

- 지연횟수가 가장 많은 달 : 12월
- 지연횟수가 많은 구간=6~8월
- **뉴욕공항의 지연은 12월과 여름철(6~8월)에 가장 빈번**
---

## 1-2 지연 출발·도착과 날씨의 관계
월별 지연 횟수와 기상 요인  
[온도, 풍속, 습도]의 상관관계 분석


In [ ]:
#| echo: true
#| code-fold: true
#| eval: true
# flights, weather 두 데이터 프레임 merge 후 사용할 컬럼만 picked2 로 변환[온도, 풍속, 습도]
#weather데이터 확인 
weather.info()
merged=pd.merge(flights, weather, how='inner')
picked2=merged[['temp','wind_speed','humid','time_hour']]
#picked2['time_hour']의 형식 변환 [object에서datetime64로 ]
picked2['time_hour'] = pd.to_datetime(picked2['time_hour'])
# 월정보 추출
picked2['month'] = picked2['time_hour'].dt.month
#월별 평균 기온, 풍속, 습도 측정
weather_inf=picked2.groupby('month')[['temp','wind_speed','humid']].mean(numeric_only=True).reset_index()
# 섭씨로 변환
def celsius(fahrenheit):
    return (fahrenheit - 32) * 5/9
weather_inf['temp'] = weather_inf['temp'].apply(celsius)
print(weather_inf)

## 도식화

In [ ]:
#| echo: true
#| code-fold: true
#| eval: true
#각 해당그래프 출력
plt.figure(figsize=(8, 4))
plt.plot(weather_inf['month'],weather_inf['temp'],
         label='temp', color='black', alpha=0.7)
plt.plot(weather_inf['month'],weather_inf['humid'],
         label='humid', color='blue', alpha=0.7)
plt.plot(weather_inf['month'],weather_inf['wind_speed'],
         label='wind_speed', color='green', alpha=0.7)
plt.xlabel('Month')
plt.ylabel('Weather_Data')
plt.title('Montly Weather Info')
plt.xticks(ticks=range(1, 13))  # 1~12월 표시
plt.legend()
plt.grid(axis='both', linestyle='--', alpha=0.7)
plt.show()

---

## 두 그래프 비교
<div style="display: flex;">
  <div style="flex: 60%;">

In [ ]:
#| code-fold: true
#| eval: true
import matplotlib.pyplot as plt
import numpy as np
# 두 개의 그래프를 위아래로 배치
fig, ax = plt.subplots(2, 1, figsize=(6, 6))  # 2개의 행으로 그래프 배치
# 첫 번째 그래프 (위쪽): 날씨 정보 그래프
ax[0].plot(weather_inf['month'], weather_inf['temp'], label='temp', color='black', alpha=0.7)
ax[0].plot(weather_inf['month'], weather_inf['humid'], label='humid', color='blue', alpha=0.7)
ax[0].plot(weather_inf['month'], weather_inf['wind_speed'], label='wind_speed', color='green', alpha=0.7)
ax[0].set_xlabel('Month')
ax[0].set_ylabel('Weather Data')
ax[0].set_title('Monthly Weather Info')
ax[0].set_xticks(range(1, 13))  # 1~12월 표시
ax[0].legend()
ax[0].grid(axis='both', linestyle='--', alpha=0.7)
# 두 번째 그래프 (아래쪽): 도착 및 출발 지연 그래프
ax[1].bar(x - width / 2, monthly_delays['arrival_delays'], width=width, label='Arrival Delays', color='blue', alpha=0.7)
ax[1].bar(x + width / 2, monthly_delays['departure_delays'], width=width, label='Departure Delays', color='red', alpha=0.7)
ax[1].set_xlabel('Month')
ax[1].set_ylabel('Number of Delays')
ax[1].set_title('Monthly Flight Delays')
ax[1].set_xticks(range(1, 13))  # 1~12월 표시
ax[1].legend()
ax[1].grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

</div>

<div style="flex: 50%; font-size: 45px;">
    풍속
    상관관계 미미
    
    습도
    여름,겨울 습도 높음
    →지연과 높은 상관
    
    온도
    지연이 많은 6, 7, 8, 12월
    온도 편차 증가→상관관계 있음
  </div>
</div>

---

 **part2**
<br>
<br>
2-1 바람 세기가 출발 지연에 영향을 미칠까? 그럼 바람 세기가 도착 지연에는 영향을 미칠까?
2-2 바람세기에 따라 출발지연과 도착지연 비교
2-3 그렇다면 바람 세기가 35m/s가 넘는다면 결항 비율이 높을까?

## 바람 세기가 출발 지연에 영향을 미칠까?


In [ ]:
#| code-fold: true
#| eval: true
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from nycflights13 import flights, weather
## 1. 바람 세기, 출발 지연시간 비교
tem = weather[['time_hour','wind_speed']]
pt = flights[['time_hour','dep_delay']]
# 두 데이터프레임을 time_hour을 기준으로 병합
tempt = pd.merge(tem,pt,how = 'inner')
#데이터프레임의 인덱스를 초기화하여 기본 인덱스로
teempt = tempt.reset_index()
#바람세기 구간 설정
bins = np.arange(0, teempt['wind_speed'].max() + 5, 5)
#구간화
teempt['wind_bin'] = pd.cut(teempt['wind_speed'], bins=bins, right=False)
#구간 지정 기준으로 데이터를 묶어 출발지연시간의 평균 
grouped = teempt.groupby('wind_bin')['dep_delay'].mean()
#데이터프레임으로 변환
df_grouped = grouped.to_frame().reset_index()
# 열 이름 지정
df_grouped.columns = ['wind_bin', 'dep_delay']  
# 구간을 문자열로 변환
df_grouped['wind_bin_str'] = df_grouped['wind_bin'].astype(str)
# 막대그래프 그리기 (Matplotlib)
bars = plt.bar(df_grouped['wind_bin_str'], df_grouped['dep_delay'], color='thistle')
bars[6].set_color('red')
plt.xlabel("Wind Bin (String)")
plt.ylabel("Dep Delay")
plt.title("Wind Bin vs Dep Delay")
plt.xticks(rotation=45) 
plt.tight_layout()
plt.show()

:::
바람세기가 30~ 35m/s일때 출발 지연이 가장 높게 나타나고<br>
오히려 35m/s를 넘으면 적게 나타나는 걸로 보여 지연이 되다못해 결항되는 것으로 추정함.

## 그럼 바람 세기가 도착 지연에는 영향을 미칠까?

In [ ]:
#| code-fold: true
#| eval: true
## 2. 바람 세기, 도착 지연시간 비교
arr = weather[['time_hour','wind_speed']]
ival = flights[['time_hour','arr_delay']]
# 두 데이터프레임을 time_hour을 기준으로 병합
arrival = pd.merge(arr,ival,how = 'inner')
#데이터프레임의 인덱스를 초기화하여 기본 인덱스로
arival = arrival.reset_index()

#동일한 바람세기 기준으로 데이터를 묶어 도착지연시간의 평균 
arv = arival.groupby('wind_speed')['arr_delay'].mean()
arv
#데이터프레임으로 변환
aarv =arv.to_frame().reset_index()
#바람세기 구간 설정
bins = np.arange(0, aarv['wind_speed'].max() + 5, 5)
#구간화
aarv['wind_bin'] = pd.cut(aarv['wind_speed'], bins=bins, right=False)
#구간 지정 기준으로 데이터를 묶어 도착착지연시간의 평균 
arvgroup = aarv.groupby('wind_bin')['arr_delay'].mean()
#데이터프레임으로 변환
arvgroup = arvgroup.to_frame().reset_index()
# 열 이름 지정
arvgroup.columns = ['wind_bin', 'arr_delay'] 
# 구간을 문자열로 변환
arvgroup['wind_bin_str'] = arvgroup['wind_bin'].astype(str)
# 막대그래프 그리기 (Matplotlib)
bars = plt.bar(arvgroup['wind_bin_str'],arvgroup['arr_delay'], color='powderblue')
bars[6].set_color('blue')
plt.xlabel("Wind Bin (String)")
plt.ylabel("Dep Delay")
plt.title("Wind Bin vs Dep Delay")
# 라벨이 겹치지 않도록 회전
plt.xticks(rotation=45)  
plt.tight_layout()
plt.show()

도착 지연 역시 바람세기가 30~ 35m/s일때 가장 높게 나타나고<br> 35m/s를 넘으면 적게 나타나는 걸로 보여 지연이 되다못해 결항되어 적게 나타나는 것으로 추정함.
## 바람세기에 따라 출발지연과 도착지연 비교

In [ ]:
#| code-fold: true
#| eval: true
#막대 두개 합치기
import numpy as np
import matplotlib.pyplot as plt
# x축 라벨
labels = df_grouped['wind_bin_str']   
val1 = df_grouped['dep_delay']
val2 = arvgroup['arr_delay']
# x축 좌표
x = np.arange(len(labels))    
# 막대 너비
width = 0.4                   
# 막대그래프
bars = plt.bar(x - width/2, val1, width=width, label='dep_delay', color='thistle')
bars[6].set_color('red')
bars = plt.bar(x + width/2, val2, width=width, label='arr_delay', color='powderblue')
bars[6].set_color('blue')
 # x축 위치에 labels 표시
plt.xticks(x, labels,rotation = 45)        
plt.legend()
plt.tight_layout()
plt.show()

바람 세기에 따라 출발 지연과 도착 지연을 비교했더니 <br>
출발 지연이 도착 지연보다 더 많은 영향을 받는 것으로 나타났으나<br> 40m/s 이후엔 도착 지연이
더 큰 영향을 받은 것으로 나타남

## 그렇다면 바람 세기가 35m/s가 넘는다면 결항 비율이 높을까?

In [ ]:
#| code-fold: true
#| eval: true
# 결항 여부를 새로운 열로 만들기: dep_time이 NaN이면 결항(True), 아니면 운항(False)
flights['cancelled'] = flights['dep_time'].isna()
de = weather[['time_hour','wind_speed']]
lay = flights[['time_hour','cancelled','dep_delay']]
delay = pd.merge(de,lay,how = 'inner')
bins = np.arange(0, delay['wind_speed'].max() + 5, 5)
delay["wind_bin"] = pd.cut(delay['wind_speed'], bins=[0, 5, 10, 15, 20, 30, 50], 
                             labels=["0-5", "5-10", "10-15", "15-20", "20-30", "30+"])
# 바람 구간별 결항 편수
cancelled_count = delay.groupby('wind_bin')['cancelled'].sum()
#해당 바람 구간에 속하는 모든 항공편(결항이든 아니든)의 건수
total_count = delay.groupby('wind_bin')['cancelled'].count()
#각 바람 구간에서 결항 건수 / 전체 운항 건수, 100을 곱해 **결항률(%)**
cancelled_rate = (cancelled_count / total_count) * 100
cancelled_rate2 = cancelled_rate.to_frame().reset_index()
cancelled_rate2['wind_bin_str'] = cancelled_rate2['wind_bin'].astype(str)
import matplotlib.pyplot as plt
bars = plt.bar(cancelled_rate2['wind_bin_str'], 
               cancelled_rate2['cancelled'], 
               width=width, label='arr_delay', color='tomato')
bars[5].set_color('green')
plt.ylabel("dep_delay(%)")
plt.xlabel("wind_speed")
plt.title("wind_speed vs dep_delay")
plt.xticks(rotation=45)
plt.tight_layout()
plt.legend()
plt.show()

35m/s이 넘으면 결항 비율이 계속해서 높아질 줄 알았으나<br> 40m/s에는 확 줄어드는 걸로 보아
바람 세기가 높아질수록 결항비율이 높아진다고 보기도 힘듦.